## Environment Setup

In [7]:
#!pip install transformers torch pandas scikit-learn


  Using cached transformers-4.44.0-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp312-none-win_amd64.whl.metadata (6.9 kB)
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.5 MB 243.2 kB/s eta 0:00:37
   -- ------------------------------------- 0.5/9.5 MB 243.2 kB/s eta 0:00:37
   -- ------------------------------------- 0.5/9.5 MB 243.2 kB/s eta 0:00:37


In [5]:
#!python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 1.3 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/1.8 MB 1.6 MB/s eta 0:00:02
   --- ------------------------------------ 0.2/1.8 MB 1.6 MB/s eta 0:00:02
   ---- ----------------------------------- 0.2/1.8 MB 1.0 MB/s eta 0:00:02
   ----- ---------------------------------- 0.3/1.8 MB 1.2 MB/s eta 0:00:02
   ------- -------------------------------- 0.3/1.8 MB 1.2 MB/s eta 0:00:02
   --------- ------------------------------ 0.5/1.8 MB 1.5 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.8 MB 1.3 MB/s eta 0:00:01
   ------------- -------------------------- 0.6/1.8 MB 1.5 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/1.8 MB 1.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/1.8 MB 1.6 MB/s eta 0:00:01
   ---------------------- ----------------- 1.0/1.8 MB 1.8 MB/s eta 0:00:01
   ----------------

 ## Import Libraries

In [17]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset, random_split
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


In [9]:
pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.3 MB 1.2 MB/s eta 0:00:02
   ------------- -------------------------- 0.8/2.3 MB 1.8 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.3 MB 1.4 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.3 MB 1.4 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 1.1 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 1.1 MB/s eta 0:00:01
   -------------------------- ------------- 1.6/2.3 MB 932.2 kB/s eta 0:00:01
   ----------------------------------- ---- 2.1/2.3 MB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


## Load Data

In [29]:
import pandas as pd

# Load your dataset
data = pd.read_excel('RAMI salon.xlsx')

# Display the first few rows of the dataframe
data.head()


,salon_name,beautician,Service_type,Review,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,RAMI salon,Rami,haircut,Thank you somuch Slon Rami... you done it real...,NaN,NaN,NaN,NaN
1,RAMI salon,Madushanka,eyebrow shaping,I had my eyebrows shaped yesterday evening by ...,NaN,NaN,NaN,NaN
2,RAMI salon,Manoj,service,Unprofessional and bad service by manoj 😔🥺can’...,NaN,NaN,NaN,NaN
3,RAMI salon,Manoj,haircut,Highly recommended.This was my second time wit...,NaN,NaN,NaN,NaN
4,RAMI salon,NaN,customer service,Bad bad service…. Very unprofessional Painful ...,NaN,NaN,NaN,NaN


In [36]:
import warnings

# Ignore specific FutureWarnings from transformers
warnings.filterwarnings('ignore', category=FutureWarning, message=".*`clean_up_tokenization_spaces`.*")


## Preprocess Data

In [38]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load your dataset
data = pd.read_excel('RAMI salon.xlsx')

# Convert all entries in 'Review' to strings and handle NaN values
data['Review'] = data['Review'].fillna('')  # Replace NaN with empty string
data['Review'] = data['Review'].astype(str)  # Ensure all data is string

# Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(texts, max_length=128):
    encoded_data = tokenizer(
        texts,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoded_data['input_ids']
    attention_masks = encoded_data['attention_mask']
    return input_ids, attention_masks

# Apply the tokenizer
input_ids, attention_masks = tokenize_data(data['Review'].tolist())

# Display shapes to verify
print(input_ids.shape)
print(attention_masks.shape)


torch.Size([58, 128])
torch.Size([58, 128])


In [42]:
# Create dummy labels if actual labels are not available
labels = torch.zeros(input_ids.size(0), dtype=torch.long)  # Example: All zeros


## Split Data into Train and Validation

In [43]:
from sklearn.model_selection import train_test_split

# Assuming labels are either correctly defined or dummy labels are created
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, labels, random_state=2018, test_size=0.1
)
train_masks, validation_masks, _, _ = train_test_split(
    attention_masks, labels, random_state=2018, test_size=0.1
)


## Create Dataloaders

In [44]:
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [46]:
import torch
print(torch.cuda.is_available())


False


In [47]:
pip install torch torchvision torchaudio

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.4 MB ? eta -:--:--
   -------- -------------

In [50]:
import warnings

warnings.filterwarnings('ignore', message="parameter name that contains `beta` will be renamed internally to `bias`")
warnings.filterwarnings('ignore', message="parameter name that contains `gamma` will be renamed internally to `weight`")


 ## Load BERT Model

In [51]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,  # Adjust based on your task
    output_attentions=False,
    output_hidden_states=False,
)
model.to("cpu")  # Explicitly move the model to CPU


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Set Up Optimizer

In [53]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


C:\Users\Micheal\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


## Train Model

In [54]:
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import numpy as np
from sklearn.metrics import f1_score

# Assuming model and tokenizer are loaded
# Assuming train_dataloader and validation_dataloader are set up

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Parameters
epochs = 4
total_steps = len(train_dataloader) * epochs
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Function to calculate the accuracy of predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Training loop
for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()        

        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs.loss
        total_train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print("  Average training loss: {0:.2f}".format(avg_train_loss))

    print("Running Validation...")
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        
        loss = outputs.loss
        logits = outputs.logits

        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        total_eval_accuracy += flat_accuracy(logits, label_ids)

    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))

print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Average training loss: 0.47
Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.35
======== Epoch 2 / 4 ========
Training...
  Average training loss: 0.34
Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.29
======== Epoch 3 / 4 ========
Training...
  Average training loss: 0.29
Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.25
======== Epoch 4 / 4 ========
Training...
  Average training loss: 0.24
Running Validation...
  Accuracy: 1.00
  Validation Loss: 0.23
Training complete!


## Save Model

In [55]:
# Saving the model after training
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_tokenizer')


('path_to_save_tokenizer\\tokenizer_config.json',
 'path_to_save_tokenizer\\special_tokens_map.json',
 'path_to_save_tokenizer\\vocab.txt',
 'path_to_save_tokenizer\\added_tokens.json')

In [58]:
# Example to filter positive reviews (assuming class '1' is positive)
recommended_reviews = [test_reviews[i] for i in range(len(test_reviews)) if test_predictions[i] == 1]

print("Recommended based on positive reviews:")
for review in recommended_reviews:
    print(review)


Recommended based on positive reviews:


In [64]:
import pandas as pd

# Example beautician data
beauticians = pd.DataFrame({
    'name': ['Rami', 'Madushanka', 'Manoj', 'Sasha'],
    'style': ['Modern', 'Traditional', 'Natural', 'Modern'],
    'interaction': ['Conversational', 'Quiet', 'Informative', 'Supportive'],
    'speed': ['Quick', 'Thorough', 'Quick', 'Thorough'],
    'personality': ['Professional', 'Friendly', 'Cheerful', 'Disciplined'],
    'average_time': ['30 min', '45 min', '30 min', '1h']
})

# User preferences
user_preferences = {
    'style': 'Natural',
    'interaction': 'Informative',
    'speed': 'Quick',
    'personality': 'Professional',
    'average_time': '45 min'
}

# Scoring function
def score_beautician(beautician, preferences):
    score = 0
    for key, value in preferences.items():
        if beautician[key] == value:
            score += 1
    return score

# Apply scoring
beauticians['score'] = beauticians.apply(score_beautician, axis=1, preferences=user_preferences)

# Filter for highest score
recommended_beauticians = beauticians[beauticians['score'] == beauticians['score'].max()]

print("Recommended Beautician(s):")
print(recommended_beauticians[['name', 'score']])


Recommended Beautician(s):
    name  score
2  Manoj      3


In [68]:
import pandas as pd
from textblob import TextBlob

# Sample beautician data
beauticians = pd.DataFrame({
    'name': ['Rami', 'Madushanka', 'Manoj', 'Sasha'],
    'style': ['Modern', 'Traditional', 'Natural', 'Modern'],
    'interaction': ['Conversational', 'Quiet', 'Informative', 'Supportive'],
    'speed': ['Quick', 'Thorough', 'Quick', 'Thorough'],
    'personality': ['Professional', 'Friendly', 'Cheerful', 'Disciplined'],
    'average_time': ['30 min', '45 min', '30 min', '1h'],
    'review': [
        "Rami provided an excellent service with a modern haircut that was quick and professional. Highly recommended!",
        "Madushanka's work on traditional makeup wasn't up to par this time, lacked the usual charm.",
        "Manoj was quite informative and friendly while providing a quick and efficient haircut. Very happy with the results!",
        "Sasha’s work was disciplined but too slow, and the support was lacking during the long session."
    ]
})

# User preferences
user_preferences = {
    'style': 'Modern',
    'interaction': 'Conversational',
    'speed': 'Quick',
    'personality': 'Professional',
    'average_time': '30 min'
}

# Function to score each beautician based on user preferences and sentiment analysis
def enhanced_score_beautician(beautician, preferences):
    score = 0
    # Attribute matching
    for key in ['style', 'interaction', 'speed', 'personality', 'average_time']:
        if beautician[key] == preferences[key]:
            score += 1
    
    # Sentiment analysis on the review
    sentiment = TextBlob(beautician['review']).sentiment.polarity
    if sentiment > 0:  # Positive sentiment boosts the score
        score += 1

    return score

# Apply the scoring function
beauticians['score'] = beauticians.apply(enhanced_score_beautician, axis=1, preferences=user_preferences)

# Recommend beauticians based on the highest score
recommended_beauticians = beauticians[beauticians['score'] == beauticians['score'].max()]
print("Recommended Beautician(s) based on Positive Reviews and Preferences:")
print(recommended_beauticians[['name', 'review', 'score']])


Recommended Beautician(s) based on Positive Reviews and Preferences:
   name                                             review  score
0  Rami  Rami provided an excellent service with a mode...      6


In [66]:
pip install textblob

   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/626.3 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/626.3 kB ? eta -:--:--
   -------------------------------------- 626.3/626.3 kB 630.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.5 MB ? eta -:--:--
   ------------- -------------------------- 0.5/1.5 MB 390.1 kB/s eta 0:00

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
label-studio-converter 0.0.58 requires nltk==3.6.7, but you have nltk 3.8.1 which is incompatible.


In [67]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Micheal\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [69]:
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from textblob import TextBlob

# Load the trained model and tokenizer
model_path = 'path_to_save_model'
tokenizer_path = 'path_to_save_tokenizer'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)

# Ensure the model and tokenizer are correctly loaded
# Example beautician data
beauticians = pd.DataFrame({
    'name': ['Rami', 'Madushanka', 'Manoj', 'Sasha'],
    'style': ['Modern', 'Traditional', 'Natural', 'Modern'],
    'interaction': ['Conversational', 'Quiet', 'Informative', 'Supportive'],
    'speed': ['Quick', 'Thorough', 'Quick', 'Thorough'],
    'personality': ['Professional', 'Friendly', 'Cheerful', 'Disciplined'],
    'average_time': ['30 min', '45 min', '30 min', '1h'],
    'review': [
        "Rami provided an excellent service with a modern haircut that was quick and professional. Highly recommended!",
        "Madushanka's work on traditional makeup wasn't up to par this time, lacked the usual charm.",
        "Manoj was quite informative and friendly while providing a quick and efficient haircut. Very happy with the results!",
        "Sasha’s work was disciplined but too slow, and the support was lacking during the long session."
    ]
})

# User preferences
user_preferences = {
    'style': 'Modern',
    'interaction': 'Conversational',
    'speed': 'Quick',
    'personality': 'Professional',
    'average_time': '30 min'
}

# Function to score each beautician based on user preferences and sentiment analysis
def enhanced_score_beautician(beautician, preferences):
    score = 0
    # Attribute matching
    for key in ['style', 'interaction', 'speed', 'personality', 'average_time']:
        if beautician[key] == preferences[key]:
            score += 1
    
    # Sentiment analysis on the review
    sentiment = TextBlob(beautician['review']).sentiment.polarity
    if sentiment > 0:  # Positive sentiment boosts the score
        score += 1

    return score

# Apply the scoring function
beauticians['score'] = beauticians.apply(enhanced_score_beautician, axis=1, preferences=user_preferences)

# Recommend beauticians based on the highest score
recommended_beauticians = beauticians[beauticians['score'] == beauticians['score'].max()]
print("Recommended Beautician(s) based on Positive Reviews and Preferences:")
print(recommended_beauticians[['name', 'review', 'score']])


Recommended Beautician(s) based on Positive Reviews and Preferences:
   name                                             review  score
0  Rami  Rami provided an excellent service with a mode...      6


In [70]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Sample data and preferences
beauticians = pd.DataFrame({
    'name': ['Rami', 'Madushanka', 'Manoj', 'Sasha'],
    'review': [
        "Rami provided an excellent service with a modern haircut that was quick and professional. Highly recommended!",
        "Madushanka's work on traditional makeup wasn't up to par this time, lacked the usual charm.",
        "Manoj was quite informative and friendly while providing a quick and efficient haircut. Very happy with the results!",
        "Sasha’s work was disciplined but too slow, and the support was lacking during the long session."
    ]
})

# Preferences with keywords (example)
preference_keywords = {
    'style': ['modern'],
    'interaction': ['conversational'],
    'speed': ['quick'],
    'personality': ['professional'],
    'average_time': ['30 min']
}

# Load the trained model and tokenizer
model_path = 'path_to_save_model'
tokenizer_path = 'path_to_save_tokenizer'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
model.eval()  # Set the model to evaluation mode

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to predict sentiment from reviews
def predict_sentiment(reviews):
    encoded_reviews = tokenizer(reviews, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encoded_reviews['input_ids'].to(device)
    attention_mask = encoded_reviews['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
    
    scores = torch.softmax(logits, dim=1)[:, 1]  # Assuming class 1 is positive sentiment
    return scores.cpu().numpy()

# Predict sentiments
beauticians['sentiment_score'] = predict_sentiment(beauticians['review'].tolist())

# Score beauticians based on user preferences and sentiment
def final_score(row, keywords):
    score = row['sentiment_score'] * 10  # Scale sentiment score
    review_lower = row['review'].lower()
    for key, words in keywords.items():
        if any(word in review_lower for word in words):
            score += 1  # Increment score for each matched keyword
    return score

beauticians['final_score'] = beauticians.apply(final_score, axis=1, keywords=preference_keywords)

# Select top recommended beauticians
recommended_beauticians = beauticians.sort_values(by='final_score', ascending=False)
print("Recommended Beautician(s) based on Reviews and User Preferences:")
print(recommended_beauticians[['name', 'final_score']])


Recommended Beautician(s) based on Reviews and User Preferences:
         name  final_score
0        Rami     5.212096
2       Manoj     3.064991
3       Sasha     2.462529
1  Madushanka     2.396789
